# Triangle Counting

In this notebook, we will count the number of trianges in our test dataset.  The NetworkX and cuGraph processes will be interleaved so that each step can be compared.


| Author Credit     |    Date    |  Update          | cuGraph Version |  Test Hardware              |
| ------------------|------------|------------------|-----------------|-----------------------------|
| Brad Rees         | 08/01/2019 | created          | 0.13            | GV100 32G, CUDA 10.2        |
|                   | 08/16/2020 | updated          | 0.15   | GV100 32G, CUDA 10.2        |
| Don Acosta        | 07/11/2022 | tested / updated | 22.08 nightly   | DGX Tesla V100 CUDA 11.5    |
| Ralph Liu         | 07/27/2022 | updated          | 22.08 nightly   | DGX Tesla V100 CUDA 11.5    |

## Introduction
Triangle Counting, as the name implies, finds the number of triangles in a graph.  Triangles are important in computing the clustering Coefficient and can be used for clustering.  


To compute the Triangle count in cuGraph use: <br>
   __df = cugraph.triangle_count(G)__

* __G__: cugraph.Graph object


   Compute the triangle (number of cycles of length three) count of the input graph.

    Parameters
    ----------
    G : cugraph.graph
        cuGraph graph descriptor, should contain the connectivity information,
        (edge weights are not used in this algorithm)

    Returns
    -------
    result : cudf.DataFrame
        GPU data frame containing 2 cudf.Series

        result['vertex']: cudf.Series
            Contains the triangle counting vertices
            
        result['counts']: cudf.Series
            Contains the triangle counting counts for that vertex

__Reference__:
    O. Green, P. Yalamanchili, L.M. Munguia,
    “Fast Triangle Counting on GPU”
    Irregular Applications: Architectures and Algorithms (IA3), 2014

#### Some notes about vertex IDs...

* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


<img src="../../img/zachary_black_lines.png" width="35%"/>

### Prep

In [3]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

# Import a built-in dataset
from cugraph.experimental.datasets import karate

In [4]:
# NetworkX libraries
import networkx as nx
import pandas as pd
from scipy.io import mmread

---
# NetworkX

In [5]:
datafile= '../../data/karate.csv'
# Read the data, this also created a NetworkX Graph 
file = open(datafile, 'rb')
df = pd.read_csv(
                file,
                delimiter=" ",
                header=None,
                names=["0", "1", "weight"],
                dtype={"0": "int32", "1": "int32", "weight": "float32"})
#Gnx = nx.read_edgelist(file)
Gnx = nx.from_pandas_edgelist(
        df,
        create_using=(nx.Graph()),
        source="0",
        target="1",
        edge_attr="weight",
    )


In [6]:
nx_count = nx.triangles(Gnx)

In [7]:
# NetworkX does not give a single count, but list how many triangles each vertex is associated with
nx_count

{1: 12,
 0: 18,
 2: 11,
 3: 10,
 4: 2,
 5: 3,
 6: 3,
 7: 6,
 8: 5,
 10: 2,
 11: 0,
 12: 1,
 13: 6,
 17: 1,
 19: 1,
 21: 1,
 31: 3,
 30: 3,
 9: 0,
 27: 1,
 28: 1,
 32: 13,
 16: 1,
 33: 15,
 14: 1,
 15: 1,
 18: 1,
 20: 1,
 22: 1,
 25: 1,
 23: 4,
 29: 4,
 24: 1,
 26: 1}

In [8]:
# To get the number of triangles, we would need to loop through the array and add up each count
count = 0
for key, value in nx_count.items():
    count = count + value

In [9]:
count

135

Let's see how that compares to cuGraph

----

# cuGraph

### Create a Graph 

In [10]:
G = karate.get_graph()
G = G.to_undirected()

RuntimeError: The datafile /home/nfs/jnke/.cugraph/datasets/karate.csv does not exist. Try get_edgelist(fetch=True) to download the datafile

### Call the Triangle Counting 

In [11]:
# Call cugraph triangles to get the triangles that each vertex is included in.
triangles_per_vertex = cugraph.triangle_count(G)

AttributeError: module 'cugraph' has no attribute 'triangle_count'

In [ ]:
triangles_per_vertex.sort_values(
                "vertex").reset_index(drop=True).rename(columns={
                    "counts": "cugraph_counts"})

Now combine them to get a count for comparison with the nx calculation

In [1]:
cugraph_triangle_results = \
                triangles_per_vertex["cugraph_counts"].sum()
cugraph_triangle_results


NameError: name 'triangles_per_vertex' is not defined

_It was that easy!_  

----


___
Copyright (c) 2019-2022, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___